Zendesk Backend Exercise
==========================

## Problem Statement

You are provided data on the stations and lines of Singapore's urban rail system, including planned additions over the next few years. Your task is to use this data to build a routing service, to help users find routes from any station to any other station on this future network.

The app should expose an API to find and display one or more routes from a specified origin to a specified destination, ordered by some efficiency heuristic. Routes should have one or more steps, like "Take [line] from [station] to [station]" or "Change to [line]". You may add other relevant information to the results.

For the line names to be displayed, using the two-letter code is sufficient.

You may use any language/framework. You may also convert the data into another format as needed.

## Data Description

The included file, StationMap.csv, describes Singapore's future rail network. Here is an extract:

```
EW23,Clementi,12 March 1988
EW24,Jurong East,5 November 1988
EW25,Chinese Garden,5 November 1988
EW26,Lakeside,5 November 1988
```

Each line in the file has 3 fields, station code, station name, and date of opening.

Note that there may be interchange stations (where train lines cross) like Buona Vista, and these are listed each time they appear in a line. For e.g., for Buona Vista, it's listed as EW21 and CC22 both. Additionally, position numbers are not always sequential; the gaps represent spaces left for future stations, and may be ignored for this exercise.

Trains can be assumed to run in both directions on every line.

## Bonus

Travel times between stations change based on the time of day, due to increased/decreased traffic and frequency in the following manner:

Peak hours (6am-9am and 6pm-9pm on Mon-Fri)
	NS and NE lines take 12 minutes per station
	All other train lines take 10 minutes
	Every train line change adds 15 minutes of waiting time to the journey

Non-Peak hours (9am-6pm on Mon-Fri, 6am-10pm on Sat & Sun)
	DT and TE lines take 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey

Night hours (10pm-6am on Mon-Sun)
	DT, CG and CE lines do not operate
	TE line takes 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey

To account for these constraints, your application should expose a new method that accepts a source, destination and start time ("YYYY-MM-DDThh:mm" format, e.g. '2019-01-31T16:00') and returns one or more routes ordered by an efficiency heuristic with clear steps involved, as well as the total travel time for each route generated. If no route is available between the selected stations, this should also be communicated clearly.

## Submission

Your submission should have clear instructions on how to run your code. You can assume that the code will be tested on a fresh installation of Ubuntu 16.04 and will not have any other editors/compilers/modules installed. Any additional packages needed need to be specified in your documentation. A running web server is not necessary, but your app may be run using one.

## Evaluation

Your submission will be judged on:
- code quality and architecture
- good programming practices such as clean code, clear comments, tests
- quality of the route suggestions
- code packaging including project structure, API design
- ease of use

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./StationMap.csv")

In [3]:
df.head(20)

,Station Code,Station Name,Opening Date
0,NS1,Jurong East,10 March 1990
1,NS2,Bukit Batok,10 March 1990
2,NS3,Bukit Gombak,10 March 1990
3,NS4,Choa Chu Kang,10 March 1990
4,NS5,Yew Tee,10 February 1996
5,NS7,Kranji,10 February 1996
6,NS8,Marsiling,10 February 1996
7,NS9,Woodlands,10 February 1996
8,NS10,Admiralty,10 February 1996
9,NS11,Sembawang,10 February 1996


In [4]:
df = df.drop_duplicates(['Station Code', 'Station Name'])

In [5]:
df.drop('Opening Date', axis=1, inplace=True)

In [6]:
df['Station Line'] = df['Station Code'].apply(lambda x: x[:2])

In [7]:
def func(x):
    x = x[2:]

    if x.isdigit():
        return int(x)
    else:
        return ord(x) - ord('A') + 1

df['Station Number'] = df['Station Code'].apply(func)

In [8]:
df.head(10)

,Station Code,Station Name,Station Line,Station Number
0,NS1,Jurong East,NS,1
1,NS2,Bukit Batok,NS,2
2,NS3,Bukit Gombak,NS,3
3,NS4,Choa Chu Kang,NS,4
4,NS5,Yew Tee,NS,5
5,NS7,Kranji,NS,7
6,NS8,Marsiling,NS,8
7,NS9,Woodlands,NS,9
8,NS10,Admiralty,NS,10
9,NS11,Sembawang,NS,11


In [9]:
lines = df['Station Line'].value_counts()
lines

DT    34
EW    33
CC    28
NS    27
TE    22
NE    16
CG     2
CE     2
Name: Station Line, dtype: int64

In [10]:
graph = {}

grouped = df.groupby('Station Line')

for name, stns in grouped:
    print(f'Processing group {name} station')

    last_stn = stns['Station Code'].iloc[0]

    for stn_no in stns['Station Code'][1:]:
        graph.setdefault(last_stn, set()).add(stn_no)
        graph.setdefault(stn_no, set()).add(last_stn)

        last_stn = stn_no

    if last_stn not in graph:
        graph[last_stn] = set()

Processing group CC station
Processing group CE station
Processing group CG station
Processing group DT station
Processing group EW station
Processing group NE station
Processing group NS station
Processing group TE station


In [11]:
graph

{'CC1': {'CC2'},
 'CC2': {'CC1', 'CC3'},
 'CC3': {'CC2', 'CC4'},
 'CC4': {'CC3', 'CC5'},
 'CC5': {'CC4', 'CC6'},
 'CC6': {'CC5', 'CC7'},
 'CC7': {'CC6', 'CC8'},
 'CC8': {'CC7', 'CC9'},
 'CC9': {'CC10', 'CC8'},
 'CC10': {'CC11', 'CC9'},
 'CC11': {'CC10', 'CC12'},
 'CC12': {'CC11', 'CC13'},
 'CC13': {'CC12', 'CC14'},
 'CC14': {'CC13', 'CC15'},
 'CC15': {'CC14', 'CC16'},
 'CC16': {'CC15', 'CC17'},
 'CC17': {'CC16', 'CC19'},
 'CC19': {'CC17', 'CC20'},
 'CC20': {'CC19', 'CC21'},
 'CC21': {'CC20', 'CC22'},
 'CC22': {'CC21', 'CC23'},
 'CC23': {'CC22', 'CC24'},
 'CC24': {'CC23', 'CC25'},
 'CC25': {'CC24', 'CC26'},
 'CC26': {'CC25', 'CC27'},
 'CC27': {'CC26', 'CC28'},
 'CC28': {'CC27', 'CC29'},
 'CC29': {'CC28'},
 'CE1': {'CE2'},
 'CE2': {'CE1'},
 'CG1': {'CG2'},
 'CG2': {'CG1'},
 'DT1': {'DT2'},
 'DT2': {'DT1', 'DT3'},
 'DT3': {'DT2', 'DT5'},
 'DT5': {'DT3', 'DT6'},
 'DT6': {'DT5', 'DT7'},
 'DT7': {'DT6', 'DT8'},
 'DT8': {'DT7', 'DT9'},
 'DT9': {'DT10', 'DT8'},
 'DT10': {'DT11', 'DT9'},
 'DT11

In [12]:
intersect_station = {}

for index, row in df.iterrows():
    stn_name = row['Station Name']
    stn_no = row['Station Code']

    intersect_station.setdefault(stn_name, []).append(stn_no)

In [13]:
intersect_station

{'Jurong East': ['NS1', 'EW24'],
 'Bukit Batok': ['NS2'],
 'Bukit Gombak': ['NS3'],
 'Choa Chu Kang': ['NS4'],
 'Yew Tee': ['NS5'],
 'Kranji': ['NS7'],
 'Marsiling': ['NS8'],
 'Woodlands': ['NS9', 'TE2'],
 'Admiralty': ['NS10'],
 'Sembawang': ['NS11'],
 'Canberra': ['NS12'],
 'Yishun': ['NS13'],
 'Khatib': ['NS14'],
 'Yio Chu Kang': ['NS15'],
 'Ang Mo Kio': ['NS16'],
 'Bishan': ['NS17', 'CC15'],
 'Braddell': ['NS18'],
 'Toa Payoh': ['NS19'],
 'Novena': ['NS20'],
 'Newton': ['NS21', 'DT11'],
 'Orchard': ['NS22', 'TE14'],
 'Somerset': ['NS23'],
 'Dhoby Ghaut': ['NS24', 'NE6', 'CC1'],
 'City Hall': ['NS25', 'EW13'],
 'Raffles Place': ['NS26', 'EW14'],
 'Marina Bay': ['NS27', 'CE2', 'TE20'],
 'Marina South Pier': ['NS28'],
 'Pasir Ris': ['EW1'],
 'Tampines': ['EW2', 'DT32'],
 'Simei': ['EW3'],
 'Tanah Merah': ['EW4'],
 'Bedok': ['EW5'],
 'Kembangan': ['EW6'],
 'Eunos': ['EW7'],
 'Paya Lebar': ['EW8', 'CC9'],
 'Aljunied': ['EW9'],
 'Kallang': ['EW10'],
 'Lavender': ['EW11'],
 'Bugis': ['EW1

In [14]:
from itertools import combinations


for stn_name, connected_stns in intersect_station.items():
    for stn1, stn2 in combinations(connected_stns, 2):
        graph.setdefault(stn1, set()).add(stn2)
        graph.setdefault(stn2, set()).add(stn1)

In [15]:
graph

{'CC1': {'CC2', 'NE6', 'NS24'},
 'CC2': {'CC1', 'CC3'},
 'CC3': {'CC2', 'CC4'},
 'CC4': {'CC3', 'CC5', 'DT15'},
 'CC5': {'CC4', 'CC6'},
 'CC6': {'CC5', 'CC7'},
 'CC7': {'CC6', 'CC8'},
 'CC8': {'CC7', 'CC9'},
 'CC9': {'CC10', 'CC8', 'EW8'},
 'CC10': {'CC11', 'CC9', 'DT26'},
 'CC11': {'CC10', 'CC12'},
 'CC12': {'CC11', 'CC13'},
 'CC13': {'CC12', 'CC14', 'NE12'},
 'CC14': {'CC13', 'CC15'},
 'CC15': {'CC14', 'CC16', 'NS17'},
 'CC16': {'CC15', 'CC17'},
 'CC17': {'CC16', 'CC19', 'TE9'},
 'CC19': {'CC17', 'CC20', 'DT9'},
 'CC20': {'CC19', 'CC21'},
 'CC21': {'CC20', 'CC22'},
 'CC22': {'CC21', 'CC23', 'EW21'},
 'CC23': {'CC22', 'CC24'},
 'CC24': {'CC23', 'CC25'},
 'CC25': {'CC24', 'CC26'},
 'CC26': {'CC25', 'CC27'},
 'CC27': {'CC26', 'CC28'},
 'CC28': {'CC27', 'CC29'},
 'CC29': {'CC28', 'NE1'},
 'CE1': {'CE2', 'DT16'},
 'CE2': {'CE1', 'NS27', 'TE20'},
 'CG1': {'CG2', 'DT35'},
 'CG2': {'CG1'},
 'DT1': {'DT2'},
 'DT2': {'DT1', 'DT3'},
 'DT3': {'DT2', 'DT5'},
 'DT5': {'DT3', 'DT6'},
 'DT6': {'DT5'

In [16]:
from collections import deque


def simple_bfs(start, end):
    queue = deque([start])
    marker = set([start])
    tracer = {}

    while queue:
        stn = queue.popleft()
        
        if stn == end:
            break

        for neighbor_stn in graph[stn]:
            if neighbor_stn not in marker:
                queue.append(neighbor_stn)
                marker.add(neighbor_stn)
                tracer[neighbor_stn] = stn

    if end not in tracer:
        return tuple()

    route = deque()

    while end != start:
        route.appendleft(end)
        end = tracer[end]

    route.appendleft(start)

    return tuple(route)

In [17]:
route = simple_bfs('CC21', 'EW12')

In [18]:
route

('CC21', 'CC20', 'CC19', 'DT9', 'DT10', 'DT11', 'DT12', 'DT13', 'DT14', 'EW12')

In [19]:
def get(row, field):
    return row[field].iloc[0]

In [20]:
def print_route(route):
    if not route:
        return

    last_row = df[df['Station Code'] == route[0]]

    for path in route[1:]:
        row = df[df['Station Code'] == path]

        last_stn_line, last_stn_name = get(last_row, 'Station Line'), get(last_row, 'Station Name')
        stn_line, stn_name = get(row, 'Station Line'), get(row, 'Station Name')

        if last_stn_line == stn_line:
            print('Take {} line from {} to {}'.format(last_stn_line, last_stn_name, stn_name))
        else:
            print('Change from {} line to {} line'.format(last_stn_line, stn_line))

        last_row = row

In [21]:
print_route(route)

Take CC line from Holland Village to Farrer Road
Take CC line from Farrer Road to Botanic Gardens
Change from CC line to DT line
Take DT line from Botanic Gardens to Stevens
Take DT line from Stevens to Newton
Take DT line from Newton to Little India
Take DT line from Little India to Rochor
Take DT line from Rochor to Bugis
Change from DT line to EW line


## Bonus

Travel times between stations change based on the time of day, due to increased/decreased traffic and frequency in the following manner:

Peak hours (6am-9am and 6pm-9pm on Mon-Fri)
	NS and NE lines take 12 minutes per station
	All other train lines take 10 minutes
	Every train line change adds 15 minutes of waiting time to the journey

Non-Peak hours (9am-6pm on Mon-Fri, 6am-10pm on Sat & Sun)
	DT and TE lines take 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey

Night hours (10pm-6am on Mon-Sun)
	DT, CG and CE lines do not operate
	TE line takes 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey

To account for these constraints, your application should expose a new method that accepts a source, destination and start time ("YYYY-MM-DDThh:mm" format, e.g. '2019-01-31T16:00') and returns one or more routes ordered by an efficiency heuristic with clear steps involved, as well as the total travel time for each route generated. If no route is available between the selected stations, this should also be communicated clearly.

In [22]:
all_lines = set(lines.keys())
all_lines

{'CC', 'CE', 'CG', 'DT', 'EW', 'NE', 'NS', 'TE'}

In [23]:
TRAVEL_TIME = 'TRAVEL_TIME'
CHANGE_LINE = 'CHANGE_LINE'

PEAK_HOURS = {
    TRAVEL_TIME: {
        ('NS', 'NE'): 12,
        tuple(all_lines - set(('NS', 'NE'))): 10
    },
    CHANGE_LINE: 15
}

NON_PEAK_HOURS = {
    TRAVEL_TIME: {
        ('DT', 'TE'): 8,
        tuple(all_lines - set(('DT', 'TE'))): 10
    },
    CHANGE_LINE: 10
}

NIGHT_HOURS = {
    TRAVEL_TIME: {
        ('DT', 'CG', 'CE'): float('inf'),
        ('TE'): 8,
        tuple(all_lines - set(('DT', 'CG', 'CE', 'TE'))): 10
    },
    CHANGE_LINE: 10
}

print(PEAK_HOURS)
print(NON_PEAK_HOURS)
print(NIGHT_HOURS)

{'TRAVEL_TIME': {('NS', 'NE'): 12, ('CE', 'CC', 'CG', 'DT', 'TE', 'EW'): 10}, 'CHANGE_LINE': 15}
{'TRAVEL_TIME': {('DT', 'TE'): 8, ('CE', 'CC', 'EW', 'CG', 'NS', 'NE'): 10}, 'CHANGE_LINE': 10}
{'TRAVEL_TIME': {('DT', 'CG', 'CE'): inf, 'TE': 8, ('EW', 'NE', 'NS', 'CC'): 10}, 'CHANGE_LINE': 10}


In [24]:
route = simple_bfs('EW27', 'EW12')
print_route(route)

Take EW line from Boon Lay to Lakeside
Take EW line from Lakeside to Chinese Garden
Take EW line from Chinese Garden to Jurong East
Take EW line from Jurong East to Clementi
Take EW line from Clementi to Dover
Take EW line from Dover to Buona Vista
Take EW line from Buona Vista to Commonwealth
Take EW line from Commonwealth to Queenstown
Take EW line from Queenstown to Redhill
Take EW line from Redhill to Tiong Bahru
Take EW line from Tiong Bahru to Outram Park
Take EW line from Outram Park to Tanjong Pagar
Take EW line from Tanjong Pagar to Raffles Place
Take EW line from Raffles Place to City Hall
Take EW line from City Hall to Bugis


In [25]:
from dataclasses import dataclass

In [26]:
@dataclass(frozen=True)
class Edge:
    u: str
    v: str
    weight: float

    def other(self, v: str):
        if self.v == v:
            return self.u
        elif self.u == v:
            return self.v
        else:
            raise ValueError

In [27]:
num_of_stations = df['Station Code'].count()

In [28]:
from collections import defaultdict


class Graph:
    def __init__(self, V):
        self.V = V
        self.adj = defaultdict(set)
        self.edges = set()

    def add_edge(self, e: Edge) -> None:
        self.adj[e.u].add(e)
        self.adj[e.v].add(e)

        self.edges.add(e)

In [29]:
def construct_graph(rule):
    graph = Graph(num_of_stations)

    grouped = df.groupby('Station Line')

    for name, stations in grouped:
        last_stn = stations['Station Code'].iloc[0]

        for stn_no in stations['Station Code'][1:]:
            edge = None

            for group, cost in rule[TRAVEL_TIME].items():
                if name in group:
                    edge = Edge(last_stn, stn_no, cost)
                    break

            graph.add_edge(edge)

            last_stn = stn_no

        if last_stn not in graph.adj:
            graph[last_stn] = set()

    for stn_name, connected_stns in intersect_station.items():
        for stn1, stn2 in combinations(connected_stns, 2):
            edge = Edge(stn1, stn2, rule[CHANGE_LINE])

            graph.add_edge(edge)

    return graph

peak_hours_graph = construct_graph(PEAK_HOURS)
non_peak_hours_graph = construct_graph(NON_PEAK_HOURS)
night_hours_graph = construct_graph(NIGHT_HOURS)

In [30]:
from typing import Tuple
from itertools import chain


class FloydWarshall:
    def __init__(self, G: Graph):
        self.G = G
        self.trace = defaultdict(lambda: defaultdict(lambda: -1))
        self.dist_to = defaultdict(lambda: defaultdict(lambda: float('inf')))

        self._execute()

    def _execute(self):
        for edge in self.G.edges:
            u, v, weight = edge.u, edge.v, edge.weight

            self.dist_to[u][v] = weight
            self.dist_to[v][u] = weight
            self.trace[u][v] = u
            self.trace[v][u] = v
            
        for stn_m in self.G.adj.keys():
            for stn_1 in self.G.adj.keys():
                for stn_2 in self.G.adj.keys():
                    optimized_cost = self.dist_to[stn_1][stn_m] + self.dist_to[stn_m][stn_2]

                    if optimized_cost < self.dist_to[stn_1][stn_2]:
                        self.dist_to[stn_1][stn_2] = optimized_cost
                        self.trace[stn_1][stn_2] = self.trace[stn_m][stn_2]

    def cost(self, start: str, end: str) -> float:
        return self.dist_to[start][end]

    def route(self, start: str, end: str) -> Tuple[str]:
        if self.trace[start][end] == -1:
            return tuple()

        path = deque([end])

        while start != end:
            end = self.trace[start][end]
            path.appendleft(end)
            
        return tuple(path)

In [31]:
peak_fw = FloydWarshall(peak_hours_graph)

In [32]:
print(peak_fw.cost('EW27', 'NE7'))
print(peak_fw.route('EW27', 'NE7'))
print_route(peak_fw.route('EW27', 'NE7'))

165
('EW27', 'EW26', 'EW25', 'EW24', 'EW23', 'EW22', 'EW21', 'CC22', 'CC21', 'CC20', 'CC19', 'DT9', 'DT10', 'DT11', 'DT12', 'NE7')
Take EW line from Boon Lay to Lakeside
Take EW line from Lakeside to Chinese Garden
Take EW line from Chinese Garden to Jurong East
Take EW line from Jurong East to Clementi
Take EW line from Clementi to Dover
Take EW line from Dover to Buona Vista
Change from EW line to CC line
Take CC line from Buona Vista to Holland Village
Take CC line from Holland Village to Farrer Road
Take CC line from Farrer Road to Botanic Gardens
Change from CC line to DT line
Take DT line from Botanic Gardens to Stevens
Take DT line from Stevens to Newton
Take DT line from Newton to Little India
Change from DT line to NE line


In [33]:
non_peak_fw = FloydWarshall(non_peak_hours_graph)

In [34]:
print(non_peak_fw.cost('EW27', 'NE7'))
print(non_peak_fw.route('EW27', 'NE7'))
print_route(non_peak_fw.route('EW27', 'NE7'))

144
('EW27', 'EW26', 'EW25', 'EW24', 'EW23', 'EW22', 'EW21', 'CC22', 'CC21', 'CC20', 'CC19', 'DT9', 'DT10', 'DT11', 'DT12', 'NE7')
Take EW line from Boon Lay to Lakeside
Take EW line from Lakeside to Chinese Garden
Take EW line from Chinese Garden to Jurong East
Take EW line from Jurong East to Clementi
Take EW line from Clementi to Dover
Take EW line from Dover to Buona Vista
Change from EW line to CC line
Take CC line from Buona Vista to Holland Village
Take CC line from Holland Village to Farrer Road
Take CC line from Farrer Road to Botanic Gardens
Change from CC line to DT line
Take DT line from Botanic Gardens to Stevens
Take DT line from Stevens to Newton
Take DT line from Newton to Little India
Change from DT line to NE line


In [35]:
night_fw = FloydWarshall(night_hours_graph)

In [36]:
print(night_fw.cost('EW27', 'NE7'))
print(night_fw.route('EW27', 'NE7'))
print_route(night_fw.route('EW27', 'NE7'))

160
('EW27', 'EW26', 'EW25', 'EW24', 'EW23', 'EW22', 'EW21', 'EW20', 'EW19', 'EW18', 'EW17', 'EW16', 'NE3', 'NE4', 'NE5', 'NE6', 'NE7')
Take EW line from Boon Lay to Lakeside
Take EW line from Lakeside to Chinese Garden
Take EW line from Chinese Garden to Jurong East
Take EW line from Jurong East to Clementi
Take EW line from Clementi to Dover
Take EW line from Dover to Buona Vista
Take EW line from Buona Vista to Commonwealth
Take EW line from Commonwealth to Queenstown
Take EW line from Queenstown to Redhill
Take EW line from Redhill to Tiong Bahru
Take EW line from Tiong Bahru to Outram Park
Change from EW line to NE line
Take NE line from Outram Park to Chinatown
Take NE line from Chinatown to Clarke Quay
Take NE line from Clarke Quay to Dhoby Ghaut
Take NE line from Dhoby Ghaut to Little India


## Heuristic route

```
Peak hours (6am-9am and 6pm-9pm on Mon-Fri)
	NS and NE lines take 12 minutes per station
	All other train lines take 10 minutes
	Every train line change adds 15 minutes of waiting time to the journey

Non-Peak hours (9am-6pm on Mon-Fri, 6am-10pm on Sat & Sun)
	DT and TE lines take 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey

Night hours (10pm-6am on Mon-Sun)
	DT, CG and CE lines do not operate
	TE line takes 8 minutes per stop
	All trains take 10 minutes per stop
	Every train line change adds 10 minutes of waiting time to the journey
```

In [37]:
from datetime import datetime, timedelta


def time(travel_time):
    return datetime.strptime(travel_time, '%Y-%m-%dT%H:%M')

In [38]:
date = time('2019-01-31T16:00')

In [39]:
date + timedelta(minutes=10)

datetime.datetime(2019, 1, 31, 16, 10)

In [40]:
date.hour

16

In [41]:
MON_TO_FRI = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday')
SAT_TO_SUN = ('Saturday', 'Sunday')


def get_rule_by_time(time):
    date = time.strftime('%A')

    if 22 < time.hour <= 6:
        return night_fw

    if date in MON_TO_FRI and 6 < time.hour <= 9 or 18 < time.hour <= 22:
        return peak_fw

    return non_peak_fw

In [42]:
def travel(travel_time, start, end):
    current_time = time(travel_time)

    cost = 0
    route = deque([start])

    while start != end:
        fw = get_rule_by_time(current_time)

        next_station = fw.route(start, end)[1]
        cost += fw.cost(start, next_station)

        print(current_time, next_station, fw.cost(start, next_station))

        current_time += timedelta(minutes=fw.cost(start, next_station))

        start = next_station
        route.append(next_station)

    print(cost)
    print_route(tuple(route))

travel('2019-01-31T17:00', 'EW27', 'NE7')

2019-01-31 17:00:00 EW26 10
2019-01-31 17:10:00 EW25 10
2019-01-31 17:20:00 EW24 10
2019-01-31 17:30:00 EW23 10
2019-01-31 17:40:00 EW22 10
2019-01-31 17:50:00 EW21 10
2019-01-31 18:00:00 CC22 10
2019-01-31 18:10:00 CC21 10
2019-01-31 18:20:00 CC20 10
2019-01-31 18:30:00 CC19 10
2019-01-31 18:40:00 DT9 10
2019-01-31 18:50:00 DT10 8
2019-01-31 18:58:00 DT11 8
2019-01-31 19:06:00 DT12 10
2019-01-31 19:16:00 NE7 15
151
Take EW line from Boon Lay to Lakeside
Take EW line from Lakeside to Chinese Garden
Take EW line from Chinese Garden to Jurong East
Take EW line from Jurong East to Clementi
Take EW line from Clementi to Dover
Take EW line from Dover to Buona Vista
Change from EW line to CC line
Take CC line from Buona Vista to Holland Village
Take CC line from Holland Village to Farrer Road
Take CC line from Farrer Road to Botanic Gardens
Change from CC line to DT line
Take DT line from Botanic Gardens to Stevens
Take DT line from Stevens to Newton
Take DT line from Newton to Little India
